# Problema 5: Producción del Sector Agro

Un terreno de 1798 hectáreas será utilizado para plantar café, soja, maíz, naranja y caña de azúcar. Cada producto genera una ganancia especifica.
  
<img src="productos_table.png" width= 300>

**Con el objetivo de maximizar la ganancia total**, indique cual es la mejor manera de distribuir las tierras para la plantación de los productos, indicando el porcentaje de terreno referente a la producción de cada producto.

**Restricciones**:
- La plantación de naranjas y caña de azúcar juntas deben ocupar un área mínima de 800 hectáreas.
- La plantación de naranjas no puede pasar un área de 10 hectáreas.
- La plantación de maíz debe ocupar un área mínima de 360 hectáreas. 
- La plantación de café debe ocupar un área mínima de 180 hectáreas. 
- Las plantaciones de maíz, soja y café juntas no pueden pasar un área de 899 hectáreas.
- Total hectáreas disponibles 1798

## Variables de decisión

* $x_{0}: \text{Hectareas de Café}$
* $x_{1}: \text{Hectareas de Soja}$
* $x_{2}: \text{Hectareas de Maiz}$
* $x_{3}: \text{Hectareas de Naranja}$
* $x_{4}: \text{Hectareas de Caña de Azucar}$

* $\text{Función Objetivo} = 120*x_{0} + 160*x_{1} + 75*x_{2} + 140*x_{3} + 140*x_{4}$.

## Restricciones

* $x_{3} + x_{4} >= 800$
* $x_{3} <= 10$
* $x_{2} >= 360$
* $x_{0} >= 180$
* $x_{0} + x_{1} + x_{2} <= 899$
* $x_{0} + x_{1} + x_{2} + x_{3} + x_{4} <= 1798$

## Instalación de Paquetes

In [260]:
# https://deap.readthedocs.io/en/master/
#!pip install deap

In [261]:
# Bibliotecas a serem utilizadas
import random
import numpy as np
from deap import algorithms, base, creator, tools

## Preparación para Optimización

1. **Función objetivo** 
2. **Variables de decisión**
3. **Operadores**

In [262]:
# Función Objetivo - Calculo de la ganancia
def objetive_function(individual):
  #codigo de la función
  #resultado retorna como tupla (result)
  return (individual[0]*120 + individual[1]*160 + individual[2]*75 + individual[3]*140 + individual[4]*140),

## Restricciones

- La plantación de naranjas y caña de azúcar juntas deben ocupar un área mínima de 800 hectáreas.
- La plantación de naranjas no puede pasar un área de 10 hectáreas.
- La plantación de maíz debe ocupar un área mínima de 360 hectáreas. 
- La plantación de café debe ocupar un área mínima de 180 hectáreas. 
- Las plantaciones de maíz, soja y café juntas no pueden pasar un área de 899 hectáreas.
- Total hectáreas disponibles 1798

* $x_{0}: \text{Hectareas de Café}$
* $x_{1}: \text{Hectareas de Soja}$
* $x_{2}: \text{Hectareas de Maiz}$
* $x_{3}: \text{Hectareas de Naranja}$
* $x_{4}: \text{Hectareas de Caña de Azucar}$

* $x_{3} + x_{4} >= 800$
* $x_{3} <= 10$
* $x_{2} >= 360$
* $x_{0} >= 180$
* $x_{0} + x_{1} + x_{2} <= 899$
* $x_{0} + x_{1} + x_{2} + x_{3} + x_{4} <= 1798$

In [263]:
#Ref: https://deap.readthedocs.io/en/master/tutorials/advanced/constraints.html?highlight=feasible#penalty-function  
def feasible(individual):
  if individual[3] > 10:
    return False
  if individual[2] < 360:
    return False
  if individual[0] < 180:
    return False
  if (individual[3] + individual[4]) < 800:
    return False
  if (individual[0] + individual[1] + individual[2]) > 899:
    return False
  if (sum(individual)) > 1798:
    return False
  return True 

In [264]:
def distance(individual):
    constraint1 = abs(individual[3] - 10)
    constraint2 = abs(individual[2] - 360)
    constraint3 = abs(individual[0] - 180)
    constraint4 = abs((individual[3] + individual[4]) - 800)
    constraint5 = abs((individual[0] + individual[1] + individual[2]) - 899)
    constraint6 = abs((sum(individual)) - 1798)
    sumatoria = constraint1 + constraint2 + constraint3 + constraint4 + constraint5 + constraint6  

    return sumatoria

In [265]:
# Creación estructura de fitness e individuo
##para problema de maximización el peso es positivo y minimización el peso es negativo
creator.create("FitnessMax", base.Fitness, weights=(1.0,)) 
creator.create("Individual", list, fitness=creator.FitnessMax)

In [266]:
toolbox = base.Toolbox()
"""
# Generador de atributos reales: nombre, función que genera cada variable, intervalo (limite superior e inferior)
toolbox.register("attr_int", random.randint,0,1000) #

# Generador de individuo
toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.attr_int, 5)
"""
toolbox.register("attr_int0", random.randint,180,1000) # Cafe
toolbox.register("attr_int1", random.randint,0,1000) # Soja
toolbox.register("attr_int2", random.randint,360,1000) # Maiz
toolbox.register("attr_int3", random.randint,0,10) # Naranja
toolbox.register("attr_int4", random.randint,0,1000) # Caña de Azucar

toolbox.register("individual", tools.initCycle, creator.Individual,
             (toolbox.attr_int0, 
              toolbox.attr_int1,
              toolbox.attr_int2, 
              toolbox.attr_int3,
              toolbox.attr_int4,),n=1)



# Generar la población
toolbox.register("population", tools.initRepeat, list, toolbox.individual)


In [267]:
#Inicializar Operadores: https://deap.readthedocs.io/en/master/api/tools.html
toolbox.register("evaluate",objetive_function)

# Crear Restricciones
toolbox.decorate("evaluate", tools.DeltaPenalty(feasible, 0, distance))

toolbox.register("mate", tools.cxTwoPoint)
toolbox.register("mutate", tools.mutGaussian, mu=0, sigma=1, indpb=0.05)
toolbox.register("select",tools.selTournament, tournsize=3)

In [268]:
pop = toolbox.population(n=100)                # inicialização da pop
hof = tools.HallOfFame(1)                      # melhor indivíduo
stats = tools.Statistics(lambda ind:ind.fitness.values) # estatísticas
stats.register("avg", np.mean)
stats.register("std", np.std)
stats.register("min", np.min)
stats.register("max", np.max)

In [269]:
print(f"Individuos de la población generada: \n{pop[:5]}")
print("len: ", len(pop))

Individuos de la población generada: 
[[487, 741, 713, 5, 677], [503, 48, 989, 1, 904], [228, 957, 513, 4, 886], [300, 406, 544, 1, 428], [337, 505, 379, 5, 1000]]
len:  100


# Ciclo evolutivo

In [270]:
pop, log =algorithms.eaSimple(population=pop, 
                              toolbox=toolbox, 
                              cxpb=0.5, 
                              mutpb=0.1, 
                              ngen=100, 
                              stats=stats, 
                              halloffame=hof, 
                              verbose=1)

gen	nevals	avg     	std    	min  	max 
0  	100   	-2304.63	847.662	-4371	-580
1  	56    	-1592.7 	656.482	-4355	-328
2  	50    	951.776 	20218.1	-1949	202085
3  	57    	20148.5 	62948.1	-1475	215020
4  	43    	54177.6 	92453  	-1042	221165
5  	54    	100258  	104984 	-683.094	221165
6  	55    	176233  	77287.2	-580    	216465
7  	64    	202891  	46727.8	-459    	217165
8  	60    	212905  	21440.1	-288.664	217165
9  	60    	210920  	30198.2	-328    	218605
10 	52    	211488  	30293  	-291.802	218605
11 	65    	208246  	42608.8	-328    	218605
12 	47    	218162  	779.911	215020  	218747
13 	54    	218574  	154.87 	217905  	218747
14 	52    	218617  	38.1833	218605  	218760
15 	53    	218647  	63.2634	218605  	218760
16 	55    	218692  	77.8731	218605  	218902
17 	47    	218752  	74.9504	218602  	218983
18 	57    	218808  	68.1771	218686  	219058
19 	71    	218863  	77.206 	218686  	219058
20 	63    	218931  	60.2884	218760  	219058
21 	53    	218976  	37.8696	218901  	219092
22 	58    	2

# Resultados

In [271]:
mejor_ind = list(hof[0])
mejor_individuo = []
for idx in mejor_ind:
    mejor_individuo.append(round(idx))
mejor_individuo

[282, 216, 400, 10, 875]

In [272]:
# Mejor Solución
print(f"Mejor Individuo: {mejor_individuo}\n")
cafe = mejor_individuo[0]
soja = mejor_individuo[1]
maiz = mejor_individuo[2]
naranja = mejor_individuo[3]
Cana_azucar = mejor_individuo[4]
total_hectareas = (sum(mejor_individuo))

print(f"Total Hectareas de Café: {cafe}")
print(f"Total Hectareas de Soja: {soja}")
print(f"Total Hectareas de Maiz: {maiz}")
print(f"Total Hectareas de Naranja: {naranja}")
print(f"Total Hectareas de Caña de Azucar: {Cana_azucar}")
print(f"Total Hectareas Cultivadas: {total_hectareas}\n")

print(f"Restricciones:")
print(f"Naranja + Caña de Azucar >= 800 : {(naranja + Cana_azucar)>=800}")
print(f"Naranja <= 10 : {(naranja)<=10}")
print(f"Maíz >= 360 : {(maiz)>=360}")
print(f"Café >= 180 : {(cafe)>=180}")
print(f"Maíz + Soja + Café <= 899 : {(maiz + soja + cafe)<= 899}")
print(f"Total hectáreas cultivadas <= 1798 : {(total_hectareas)<= 1798}\n")

#validacióon del mejor individuo
print(f"Lucro Total: ${round(objetive_function(mejor_individuo)[0])}")

Mejor Individuo: [282, 216, 400, 10, 875]

Total Hectareas de Café: 282
Total Hectareas de Soja: 216
Total Hectareas de Maiz: 400
Total Hectareas de Naranja: 10
Total Hectareas de Caña de Azucar: 875
Total Hectareas Cultivadas: 1783

Restricciones:
Naranja + Caña de Azucar >= 800 : True
Naranja <= 10 : True
Maíz >= 360 : True
Café >= 180 : True
Maíz + Soja + Café <= 899 : True
Total hectáreas cultivadas <= 1798 : True

Lucro Total: $222300


In [273]:
# Individuo Viable?
feasible(mejor_individuo)

True